# Libraries

In [1]:
import ast
import math
import os
import gc

import cv2
from pathlib import Path

from PIL import Image, ImageSequence
import numpy as np
from numpy.fft import fft2, fftshift, ifft2, ifftshift
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, argrelextrema
# from skfda import FDataGrid
from numpy import unravel_index
import statistics
import heapq
import pandas as pd

# import tensorflow as tf
# from tensorflow.keras import layers, models

import itertools

from scipy.ndimage import gaussian_filter
from scipy.optimize import curve_fit
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.patches as patches

# Functions

## Smoothing

In [2]:
def moving_average(signal, window_size=5):
    return np.convolve(signal, np.ones(window_size) / window_size, mode='same')

In [3]:
def merge_peaks_by_distance(peaks, min_distance):
    merged_peaks = []
    i = 0
    while i < len(peaks) - 1:
        if abs(peaks[i] - peaks[i+1]) <= min_distance:
            merged_peak = (peaks[i] + peaks[i+1]) // 2
            merged_peaks.append(merged_peak)
            i += 2  
        else:
            merged_peaks.append(peaks[i])
            i += 1

    if i == len(peaks) - 1:
        merged_peaks.append(peaks[i])
    return merged_peaks


## Circle functions

In [4]:
def circle_radius_fun(peaks_x,peaks_y,ring_index,ccol,crow):
    # Adjusted peak selection to use closest peaks around ccol
    closest_peak_x = min(peaks_x, key=lambda x: abs(x-ccol))
    closest_peak_y = min(peaks_y, key=lambda y: abs(y-crow))

    # Using peak indices to find distances to closest peaks
    # peak_index_x = peaks_x.tolist().index(closest_peak_x)
    # peak_index_y = peaks_y.tolist().index(closest_peak_y)
    peak_index_x=peaks_x.index(closest_peak_x)
    peak_index_y=peaks_y.index(closest_peak_y)

    if peak_index_x > 0:
        if len(peaks_x)==peak_index_x+1:
            peak_index_x-=1
        rl_x = ccol - peaks_x[peak_index_x - ring_index]
        rr_x = peaks_x[peak_index_x + ring_index] - ccol

    if peak_index_y > 0:
        if len(peaks_y)==peak_index_y+1:
            peak_index_y-=1
        ru_y = crow - peaks_y[peak_index_y - ring_index]
        rl_y = peaks_y[peak_index_y + ring_index] - crow


    return round((rl_x + rr_x + ru_y + rl_y) / 4., 5)

In [5]:
def circle_change_limiter(circle_radius,prev_circle_radius,allowed_change,ring_index,peaks_x,peaks_y,ccol,crow):
    if prev_circle_radius is not None and circle_radius > prev_circle_radius * (1+allowed_change) and ring_index != 1:
        ring_index=ring_index-1
        # ccol=prev_ccol
        # crow=prev_crow
        print(f'{circle_radius} high')
        circle_radius=circle_radius_fun(peaks_x,peaks_y,ring_index,ccol,crow)
    elif prev_circle_radius is not None and circle_radius < prev_circle_radius * (1-allowed_change):
        print(prev_circle_radius * 1-allowed_change)
        ring_index=ring_index+1
        # ccol=prev_ccol
        # crow=prev_crow
        print('low')
        circle_radius=circle_radius_fun(peaks_x,peaks_y,ring_index,ccol,crow)
    return circle_radius,ring_index

In [6]:
def get_center_from_peaks(peaks, current_center):
    valid_peaks = [peak for peak in peaks if np.abs(peak - current_center) >= 10]
    left_peaks = np.array([peak for peak in valid_peaks if peak < current_center])
    right_peaks = np.array([peak for peak in valid_peaks if peak > current_center])

    if len(left_peaks) == 0 or len(right_peaks) == 0:
        return current_center, 0

    left_peak = left_peaks[-1] 
    right_peak = right_peaks[0] 

    refined_center = (left_peak + right_peak) / 2
    

    distance_between_peaks = np.abs(right_peak - left_peak)

    return refined_center, distance_between_peaks

In [7]:
def adaptive_movement_threshold(prev_ccol, prev_crow, ccol, crow, prev_movement, allowed_change=5):
    # movement
    if prev_ccol != None and prev_crow!= None:
        distance = np.linalg.norm([ccol - prev_ccol, crow - prev_crow])
        print(distance)
        
        # if the center moves in the video, increase allowance
        if distance > prev_movement:
            allowed_change = min(distance, allowed_change * 2)
        
        # Allow movement if it is within the allowed range
        if distance <= allowed_change:
            return ccol, crow, distance
        else:
            # Reject large shifts
            return prev_ccol, prev_crow, prev_movement
    else:
        return ccol,crow,prev_movement


In [8]:
def get_center_from_minimums(minimums, current_center,min_depth=0.25):
    valid_minimums = [minm for minm in minimums if np.abs(minm - current_center) >= 10]
    left_minimums = np.array([minm for minm in valid_minimums if minm < current_center])
    right_minimums = np.array([minm for minm in valid_minimums if minm > current_center])

    if len(left_minimums) == 0 or len(right_minimums) == 0:
        return current_center, 0

    left_minimum = left_minimums[-2] 
    right_minimum = right_minimums[1]  

    refined_center = (left_minimum + right_minimum) // 2

    distance_between_minimums = np.abs(right_minimum - left_minimum)

    return refined_center, distance_between_minimums


In [9]:
def draw_circle(image,ccol,crow,circle_radius):
    cross_length=10

    circle_img = cv2.circle(image.copy(), (ccol, crow), int(circle_radius), (255, 0, 0), 2)
    cv2.line(circle_img,(ccol - cross_length, crow), (ccol + cross_length, crow), (255, 255, 255), 1)
    cv2.line(circle_img, (ccol, crow - cross_length), (ccol, crow + cross_length), (255, 255, 255), 1)
    
    return circle_img

## Image extraction and exploration

In [10]:
def image_background_brightness(image):
        top_left = image[0:5, 0:5]
        top_right = image[0:5, -5:]
        bottom_left = image[-5:, 0:5]
        bottom_right = image[-5:, -5:]
        brightness = np.concatenate((top_left.flatten(), top_right.flatten(), 
                                  bottom_left.flatten(), bottom_right.flatten()))
        # brightness=top_left+top_right+bottom_left+bottom_right

        return [np.mean(brightness),np.median(brightness),statistics.mode(brightness)]

In [11]:
def tiff_to_png(input_tiff, input_path, output_path):
    try:
        sq = Image.open(os.path.join(input_path, input_tiff))
        for i, img in enumerate(ImageSequence.Iterator(sq)):
            output = os.path.join(output_path, f"frame_{i:06d}.png")
            img.save(output)
    finally:
        # print("PNG extraction done")
        sq.close()

In [12]:
def get_first_png(input_tiff, input_path):
    try:
        sq = Image.open(os.path.join(input_path, input_tiff))
        first_frame = next(ImageSequence.Iterator(sq))
        first_frame_np = np.array(first_frame)
        if first_frame_np.ndim == 2:
            return first_frame_np
        else:
            return cv2.cvtColor(first_frame_np, cv2.COLOR_RGB2GRAY)
    finally:
        # print("PNG extraction done")
        sq.close()

In [13]:
def pad_image(image, target_size=(512, 512), padding_value=0):
    old_height, old_width = image.shape

    # Padding sizes
    top_pad = (target_size[0] - old_height) // 2
    bottom_pad = target_size[0] - old_height - top_pad
    left_pad = (target_size[1] - old_width) // 2
    right_pad = target_size[1] - old_width - left_pad

    padded_image = np.pad(image, ((top_pad, bottom_pad), (left_pad, right_pad)), mode='constant', constant_values=padding_value)

    return padded_image

In [14]:
def filtered_image(ftimage,crow,ccol,r_in,r_out):
    rows, cols = ftimage.shape
    mask=np.zeros((rows,cols),dtype=np.uint8)

    x,y=np.ogrid[:rows,:cols]
    mask_area = np.logical_and(((x - crow)**2 + (y - ccol)**2 >= r_in**2),
                            ((x - crow)**2 + (y - ccol)**2 <= r_out**2))
    mask[mask_area] = 1

    m_app_ftimage = ftimage * mask
    i_ftimage = ifftshift(m_app_ftimage)
    result_img = ifft2(i_ftimage)
    tmp = np.log(np.abs(result_img) + 1)
    
    return tmp, result_img

In [15]:
def intensity_profile(source,crow,ccol):
    central_line_y=source[crow,:]
    central_line_x=source[:,ccol]

    smoothed_central_line_y =moving_average(central_line_y, window_size=4)
    smoothed_central_line_x = moving_average(central_line_x, window_size=4)

    intensity={
        'central_line_y':central_line_x,
        'central_line_x':central_line_y,

        'smoothed_central_line_y' : smoothed_central_line_y,
        'smoothed_central_line_x': smoothed_central_line_x,

        'peaks_y' : get_subpixel_peak_com(argrelextrema(central_line_y, np.greater)[0], central_line_y),
        'peaks_x' : get_subpixel_peak_com(argrelextrema(central_line_x, np.greater)[0], central_line_x),
        
        'minimums_y' : get_subpixel_minimum_com(argrelextrema(smoothed_central_line_y, np.less)[0],central_line_y),
        'minimums_x' : get_subpixel_minimum_com(argrelextrema(smoothed_central_line_x, np.less)[0],central_line_x)
    }
    return intensity 

In [16]:
def diagonal_intensity_profile(source, crow, ccol):
    # Compute the main diagonal and anti-diagonal
    main_diagonal = np.diagonal(source, offset=int(crow - ccol))
    # fliplr flips the image horizontaly, then get the antidiagonal
    anti_diagonal = np.diagonal(np.fliplr(source), offset=int((ccol + crow - (source.shape[1] - 1))))

    smoothed_main_diagonal = moving_average(main_diagonal, window_size=4)
    smoothed_anti_diagonal = moving_average(anti_diagonal, window_size=4)

    intensity = {
        'main_diagonal': main_diagonal,
        'anti_diagonal': anti_diagonal,
        'smoothed_main_diagonal': smoothed_main_diagonal,
        'smoothed_anti_diagonal': smoothed_anti_diagonal,
        'peaks_main_diag': get_subpixel_peak_com(argrelextrema(main_diagonal, np.greater)[0], main_diagonal),
        'peaks_anti_diag': get_subpixel_peak_com(argrelextrema(anti_diagonal, np.greater)[0], anti_diagonal),
        'minimums_main_diag': get_subpixel_minimum_com(argrelextrema(smoothed_main_diagonal, np.less)[0], main_diagonal),
        'minimums_anti_diag': get_subpixel_minimum_com(argrelextrema(smoothed_anti_diagonal, np.less)[0], anti_diagonal),
    }

    return intensity

In [17]:
def get_center_from_diagonals(source, ccol, crow):
    diagonal_intensity = diagonal_intensity_profile(source, crow, ccol)

    # get current center peak in diagonal intensity profile
    offset = crow - ccol # starting point
    if offset >= 0: 
        start_row_main = offset
        start_col_main = 0
    else:
        start_row_main = 0
        start_col_main = -offset
    coords_main = crow - start_row_main    
    
    main_valid_peaks = [peak for peak in diagonal_intensity['peaks_main_diag'] if np.abs(peak - coords_main) >= 10]
    left_diag_peaks = np.array([peak for peak in main_valid_peaks if round(peak,0) < coords_main])
    right_diag_peaks = np.array([peak for peak in main_valid_peaks if round(peak,0) > coords_main])

    # get current center peak in antidiagonal intensity profile
    offset = ccol + crow - (source.shape[1] - 1) # starting point
    if offset >= 0:
        start_row_anti = offset
        start_col_anti = 0
    else:
        start_row_anti = 0
        start_col_anti = -offset 
    coords_anti = crow - start_row_anti

    anti_valid_peaks = [peak for peak in diagonal_intensity['peaks_main_diag'] if np.abs(peak - coords_anti) >= 10]
    left_anti_diag_peaks = np.array([peak for peak in anti_valid_peaks if round(peak,0) < coords_anti])
    right_anti_diag_peaks = np.array([peak for peak in anti_valid_peaks if round(peak,0) > coords_anti])

    if len(left_diag_peaks) == 0 or len(right_diag_peaks) == 0 or len(left_anti_diag_peaks) == 0 or len(right_anti_diag_peaks) == 0:
        return crow, ccol  # Return current center if no valid peaks

    # Peaks closest to the center
    left_diag_peak = left_diag_peaks[-1]
    right_diag_peak = right_diag_peaks[0]

    left_anti_diag_peak = left_anti_diag_peaks[-1]
    right_anti_diag_peak = right_anti_diag_peaks[0]

    # average closest peaks for center
    refined_diag_center = (left_diag_peak + right_diag_peak) / 2
    refined_anti_diag_center = (left_anti_diag_peak + right_anti_diag_peak) / 2

    crow_main = start_row_main + refined_diag_center
    ccol_main = start_col_main + refined_anti_diag_center

    crow_anti = start_row_anti + refined_anti_diag_center
    ccol_anti = start_col_anti + refined_anti_diag_center

    crow_refined=(crow_main+crow_anti)/2
    ccol_refined=(ccol_main+ccol_anti)/2

    return crow_refined,ccol_refined


## Subpixel peak

In [18]:
def gaussian(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

In [19]:
def get_subpixel_peak_gaussian(peaks, intensity_profile):
    refined_peaks = []
    for peak in peaks:
        if peak > 0 and peak < len(intensity_profile) - 1:
            # Gaussian curve
            x_vals = np.arange(peak - 1, peak + 2)
            y_vals = intensity_profile[peak - 1:peak + 2]
            try:
                popt, _ = curve_fit(gaussian, x_vals, y_vals, p0=[y_vals[1], peak, 1])
                refined_peaks.append(popt[1])
            except RuntimeError:
                refined_peaks.append(peak)
    return refined_peaks

In [20]:
def get_subpixel_peak_com(peaks, intensity_profile):
    refined_peaks = []
    for peak in peaks:
        if 0 < peak < len(intensity_profile) - 1:
            x = np.array([peak - 1, peak, peak + 1], dtype=float)
            y = intensity_profile[x.astype(int)]
            refined_peak = np.sum(x * y) / np.sum(y)
            refined_peaks.append(refined_peak)
        else:
            refined_peaks.append(peak)
    return refined_peaks


In [21]:
def get_subpixel_minimums_parabola(min_indices, data):
    subpixel_mins = []
    
    for idx in min_indices:
        
        if idx > 0 and idx < len(data) - 1:
            y1, y2, y3 = data[idx - 1], data[idx], data[idx + 1]
            
            # Parabolic interpolation
            numerator = (y1 - y3)
            denominator = (y1 - 2 * y2 + y3)
            if denominator != 0:
                subpixel_offset = 0.5 * numerator / denominator
            else:
                subpixel_offset = 0 
        
            subpixel_idx = idx + subpixel_offset
            subpixel_mins.append(subpixel_idx)
        else:
            subpixel_mins.append(idx)
    
    return np.array(subpixel_mins)

In [22]:
def get_subpixel_minimum_com(minimums, intensity_profile):
    refined_peaks = []
    for peak in minimums:
        if 0 < peak < len(intensity_profile) - 1:
            x = np.array([peak - 1, peak, peak + 1], dtype=float)
            y = intensity_profile[x.astype(int)]
            refined_peak = np.sum(x * y) / np.sum(y)
            refined_peaks.append(refined_peak)
        else:
            refined_peaks.append(peak)
    return refined_peaks


## peak refinement

In [23]:
def peaks_by_minimums(minimums,peaks):
    refined_peaks=[]
    peaks_and_minimums=zip(peaks,minimums)
    # print(peaks_and_minimums)
    for peak,minimum in peaks_and_minimums:
        if 0<peak<len(peaks)-1:
            refined_peak=(minimum-1+minimum+1)/2
        else:
            refined_peaks.append(peak)
    # print(refined_peaks)
    return refined_peaks            

# Main

In [24]:
def ring_image(r_in,r_out,ring_index,allowed_change,output_path,filename,result_folder):

    # centrui naudojamas kitas bandpass filtras, nes 36 geriau aptinka centra nei 37
    r_in_center=3
    r_out_center=6

    r_in_radius=3
    r_out_radius=6


    file_path = os.path.join(output_path, filename)
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    brightness=image_background_brightness(image)

    fig, ax = plt.subplots(1,2,figsize=(15,10))

    imageBright = cv2.convertScaleAbs(image,10,1.5)

    ax[0].imshow(image, cmap='gray')

    ftimage = fft2(imageBright)
    ftimage = fftshift(ftimage)

    rows, cols = ftimage.shape
    crow, ccol = rows // 2, cols // 2

    # applying filter for center
    tmp_center, result_img_center = filtered_image(ftimage,crow,ccol,r_in_center,r_out_center)

    # applying filter for radius
    tmp_radius, result_img_radius = filtered_image(ftimage,crow,ccol,r_in_radius,r_out_radius)

    # crow, ccol = unravel_index(tmp.argmax(), tmp.shape)
    crow, ccol = unravel_index(tmp_center.argmax(), result_img_center.shape)
    
    
    # intensity profile for circle center
    intensity_center=intensity_profile(tmp_center,crow,ccol)

    # intensity profile for circle radius
    intensity_radius=intensity_profile(tmp_radius,crow,ccol)

    # intensity profile of original image
    intensity_orig = intensity_profile(imageBright,crow,ccol)

    

    if len(intensity_center['peaks_x']) > 1 and len(intensity_center['peaks_y']) > 1:
        ccol, x_distance = get_center_from_peaks(intensity_center['peaks_y'], ccol)
        crow, y_distance = get_center_from_peaks(intensity_center['peaks_x'], crow)

        # getting the center based from diagonals
        crow_diag,ccol_diag = get_center_from_diagonals(tmp_center, ccol, crow)
        

        ccol=(ccol+ccol_diag)/2
        crow=(crow+crow_diag)/2

        refined_peaks_x_radius=peaks_by_minimums([(a+b)/2 for a,b in zip(intensity_radius['minimums_x'],intensity_orig['minimums_x'])],[(a+b)/2 for a,b in zip(intensity_radius['peaks_x'],intensity_orig['peaks_x'])])
        refined_peaks_y_radius=peaks_by_minimums([(a+b)/2 for a,b in zip(intensity_radius['minimums_y'],intensity_orig['minimums_y'])],[(a+b)/2 for a,b in zip(intensity_radius['peaks_y'],intensity_orig['peaks_y'])])
        # refined_peaks_x_radius=peaks_by_minimums(intensity_radius['minimums_x'],intensity_radius['peaks_x'])
        # refined_peaks_y_radius=peaks_by_minimums(intensity_radius['minimums_y'],intensity_radius['peaks_y'])

        circle_radius = circle_radius_fun(refined_peaks_x_radius,refined_peaks_y_radius,ring_index,ccol,crow)
        
        ccol = round(ccol, 5)
        crow = round(crow, 5)
        circle_radius = round(circle_radius, 5)

        # circle_center=(crow,ccol)
        circle_center=(ccol,crow)

        prev_circle_radius=circle_radius

        circle = patches.Circle(circle_center, circle_radius, edgecolor='black', facecolor='none', linewidth=1)
        ax[0].add_patch(circle)

        
        ax[0].set_xlim(0, 300) 
        ax[0].set_ylim(0, 300)  
        ax[0].set_aspect('equal', 'box') 

        cross_size = 20 
        ax[0].plot([circle_center[0] - cross_size, circle_center[0] + cross_size], 
                    [circle_center[1], circle_center[1]], color='blue', linewidth=2)  
        ax[0].plot([circle_center[0], circle_center[0]], 
                [circle_center[1] - cross_size, circle_center[1] + cross_size], color='blue', linewidth=2) 

        # fig.savefig(result_folder)
    
        plt.clf()
        plt.cla()
        plt.close('all')
        del image
        del ftimage, tmp_center,tmp_radius, result_img_center,result_img_radius
        gc.collect()
        
        return pd.DataFrame({
        'center_y(ccol)': [ccol],
        'center_x(crow)': [crow],
        'circle_radius': [circle_radius]
        })


    return None

In [25]:
def ring_search(test_number,r_in,r_out,ring_index,allowed_change,input_path,input_tiff,output_base_path,output_path,result_folder,excel_path):
    
    # output_path = Path(output_base_path) / Path(input_tiff).stem
    output_path=Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)
    # tiff_to_png(input_tiff, input_path, output_path)

    Path(result_folder).mkdir(parents=True, exist_ok=True)
    
    filenames=os.listdir(output_path)

    df_list=[]
    i=0
    # with ThreadPoolExecutor(max_workers=10) as executor:
    #      futures = [executor.submit(ring_image, r_in, r_out, ring_index, allowed_change, output_path, filename) for filename in filenames]
    #      for future in as_completed(futures):
    #         i+=1
    #         result = future.result()
    #         if result is not None: 
    #             df_list.append(result)
    #         if i%100==0:
    #             print(i)
    for filename in filenames:
        i+=1
        df_list.append(ring_image(r_in, r_out, ring_index, allowed_change, output_path, filename,result_folder))
        if i%100==0:
            print(i)

    final_df = pd.concat(df_list, ignore_index=True)
    # final_df.to_excel(f"ExcelsPrecision/Video_{test_number}_{r_in}{r_out}_{ring_index}_comapprox.xlsx")
    final_df.to_excel(excel_path)
    # final_df.to_excel(f"ExcelsPrecision/VideoLarge_object{test_number}_{r_in}{r_out}_{ring_index}.xlsx")

    print("All images processed and saved in the results folder.")

# tests

## large

In [26]:
# test='1'
# input_path='Tiffs_Validation'
# input_tiff=f'Test_Large{test}.tif'
# # input_tiff=f'Video{test}.tif'
# r_in = 3
# r_out = 7
# ring_index = 1
# # brightness=image_background_brightness(get_first_png(input_tiff, input_path))
# params={
#     'test_number':test,
#     'r_in':r_in,
#     'r_out':r_out,
#     'ring_index':ring_index,
#     'allowed_change':1,
#     'input_path':input_path,
#     'input_tiff':input_tiff,
#     'output_base_path':f'Results_Validation/Video_object{test}',
#     'output_path':f'Results_Validation/VideoLarge/cropped_object_{test}',
#     'result_folder' : f"Results_Validation/{test}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}",
#     'excel_path' : f'ExcelsPrecision/VideoLarge_object{test}_{r_in}{r_out}_{ring_index}_comapprox.xlsx'
# }

# ring_search(**params)

## validation

In [28]:
filters = [[3,7]]
for filter in filters:
    for test in range(1,11):
        input_path='../../Tiffs_Validation'
        # input_tiff=f'../../Test_Large{test}.tif'
        input_tiff=f'../../Video{test}.tif'
        r_in = filter[0]
        r_out = filter[1]
        ring_index = 1
        # brightness=image_background_brightness(get_first_png(input_tiff, input_path))
        params={
            'test_number':test,
            'r_in':r_in,
            'r_out':r_out,
            'ring_index':ring_index,
            'allowed_change':1,
            'input_path':input_path,
            'input_tiff':input_tiff,
            'output_base_path':f'../../Results_Validation/Video{test}',
            'output_path':f'../../Results_Validation/Video{test}/Video{test}',
            'result_folder' : f"../../Results_Validation/{test}/" + Path(input_tiff).stem + f"_Results_bw{r_in}{r_out}_{ring_index}",
            'excel_path' : f'../ExcelsTesting/Video{test}_{r_in}{r_out}_{ring_index}_allInOne.xlsx'
        }
        try:
            ring_search(**params)
        except Exception as e:
            print(e)
            continue
        # ring_search(**params)

All images processed and saved in the results folder.
All images processed and saved in the results folder.
All images processed and saved in the results folder.
All images processed and saved in the results folder.
cannot access local variable 'ru_y' where it is not associated with a value
All images processed and saved in the results folder.
All images processed and saved in the results folder.
All images processed and saved in the results folder.
All images processed and saved in the results folder.
All images processed and saved in the results folder.
